In [14]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from tensorflow.keras.utils import plot_model
import datetime as dt


# dimensions of images.
img_width, img_height = 224, 224

train_data_dir = '../data/train'
validation_data_dir = '../data/validation'
nb_train_samples = 222 # number of training images with a 80/20 train-test-split
nb_validation_samples = 55 # number of validationn images with a 80/20 train-test-split
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(3))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])


train_datagen = ImageDataGenerator(
    rotation_range=20,
    shear_range=0.2, 
    zoom_range=0.2,
    channel_shift_range=20,
    horizontal_flip=True,
    rescale=1./255,
    validation_split=0.2
    )

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training',
    interpolation = 'lanczos',
    shuffle = True
    )


validation_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation',
    interpolation = 'lanczos',
    shuffle = False)

time = dt.datetime.now().strftime("%Y_%m_%d_%H_%M_%S")

tbCallBack = TensorBoard(
    log_dir='./logs/'+time,
    histogram_freq=10, 
    write_graph=True, 
    write_images=True)

esCallback = EarlyStopping(monitor='val_accuracy',
                           verbose=10,
                           patience=10,
                           mode='max',
                           restore_best_weights=True)

model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size,
    callbacks=[tbCallBack, esCallback]
    )

model.save('../models/app_ban_ora_selftrained')


Found 222 images belonging to 3 classes.
Found 55 images belonging to 3 classes.
Epoch 1/50
13/13 [==============================] - 31s 2s/step - loss: 1.2281 - accuracy: 0.5049 - val_loss: 1.0386 - val_accuracy: 0.4375
Epoch 2/50
13/13 [==============================] - 28s 2s/step - loss: 0.8609 - accuracy: 0.6068 - val_loss: 1.0704 - val_accuracy: 0.7708
Epoch 3/50
13/13 [==============================] - 28s 2s/step - loss: 0.8998 - accuracy: 0.6408 - val_loss: 0.7967 - val_accuracy: 0.7083
Epoch 4/50
13/13 [==============================] - 29s 2s/step - loss: 0.6307 - accuracy: 0.7233 - val_loss: 0.6261 - val_accuracy: 0.8333
Epoch 5/50
13/13 [==============================] - 29s 2s/step - loss: 0.7139 - accuracy: 0.7330 - val_loss: 0.5581 - val_accuracy: 0.7292
Epoch 6/50
13/13 [==============================] - 30s 2s/step - loss: 0.5840 - accuracy: 0.7621 - val_loss: 0.6681 - val_accuracy: 0.6875
Epoch 7/50
13/13 [==============================] - 29s 2s/step - loss: 0.4790 

In [17]:
# evaluation of the model
X_train, y_train = train_generator.next()
X_test, y_test = validation_generator.next()
results_train = model.evaluate(X_train, y_train)
print("train loss, train acc:", results_train)
results_test = model.evaluate(X_test, y_test)
print("test loss, test acc:", results_test)

1/1 [==============================] - 0s 790us/step - loss: 0.1402 - accuracy: 0.9375
train loss, train acc: [0.1401657611131668, 0.9375]
1/1 [==============================] - 0s 938us/step - loss: 0.0107 - accuracy: 1.0000
test loss, test acc: [0.010707075707614422, 1.0]
